<a href="https://colab.research.google.com/github/desarrrolloweb/coursera-test/blob/master/Copia_de_Untitled1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Establezca una dirección IP y configure DHCP

Establecer una dirección IP en una computadora con Windows es una tarea que la mayoría de los profesionales de TI ya se han comprometido con la memoria muscular. Haga clic en esto, haga clic con el botón derecho en eso, haga clic en esto, escriba la dirección IP, la máscara de subred, la puerta de enlace predeterminada y los servidores DNS ... ¡listo! Es fácil de hacer, pero es una molestia cuando es parte de un flujo de trabajo que debe automatizar de alguna manera. Para automatizar la configuración de una dirección IP, podemos usar PowerShell para que sea rápido y fácil

PowerShell incluye algunos cmdlets que nos permiten encontrar la dirección IP de una máquina y cambiarla a cualquier configuración que deseemos. Veamos algunos escenarios.

## Establecer una dirección IP estática para DHCP

Una de las tareas más fáciles de realizar es establecer una NIC en DHCP. Con los cmdlets Set-NetIPInterface y Remove-NetRoute , podemos hacer que suceda. Comenzamos con una configuración de IP similar a la siguiente.

![texto alternativo](https://4sysops.com/wp-content/uploads/2018/05/Setting-the-current-IP-address-in-the-GUI.png)

Para configurar esta NIC en DHCP, podemos usar el comando Set-NetIpInterface . Esto eliminará la dirección IP y la máscara de subred.

In [0]:
 Set-NetIPInterface   -InterfaceAlias   'Ethernet 2'   -Dhcp   Enabled

Sin embargo, tras una inspección adicional, la puerta de enlace predeterminada aparece atenuada en las propiedades de TCP / IP pero aún permanece. Tenemos que usar un comando diferente para eliminar esto, llamado Remove-NetRoute .

In [0]:
 PS >   Get - NetIPAddress   - InterfaceAlias   'Ethernet 2'   |   Remove - NetRoute 
 
 

In [0]:
Confirm 
 Are  you  sure  you  want  to   perform  this   action ? 
 Performing  operation   "Remove"   on  Target   "NetRoute -DestinationPrefix 169.254.114.202/32 -InterfaceIndex 8 -NextHop 0.0.0.0 -Store Active" 
 [ Y ]   Yes    [ A ]   Yes  to   All    [ N ]   No    [ L ]   No  to   All    [ S ]   Suspend    [ ? ]   Help   ( default   is   "Y" ) :   a 

Esto establece la NIC en DHCP sin restos de la puerta de enlace predeterminada anterior.

## Establecer una dirección IP estática desde DHCP

Al pasar de DHCP a estático, los cmdlets de PowerShell tratan esto como una "nueva" dirección IP, por lo tanto, el uso del cmdlet New-NetIpAddress . Para usar esto, deberá hacer referencia a la dirección IP actual y canalizarla a New-NetIpAddress utilizando la dirección IP esperada, la longitud del prefijo de la máscara de subred y la puerta de enlace predeterminada.

In [0]:
 PS   C : >   Get - NetIpAddress   - InterfaceAlias   'Ethernet 2'   |   New - NetIpAddress   IpAddress   192.168.1.10   - PrefixLength   24   - DefaultGateway   192.168.1.1 
 
 

In [0]:
IPAddress   :   192.168.1.10   InterfaceIndex   :   8   InterfaceAlias   :   Ethernet   2   AddressFamily   :   IPv4  Type   :   Unicast  PrefixLength   :   24   PrefixOrigin   :   Manual  SuffixOrigin   :   Manual  AddressState   :   Tentative  ValidLifetime   :   Infinite   ( [ TimeSpan ] :: MaxValue )   PreferredLifetime   :   Infinite   ( [ TimeSpan ] :: MaxValue )   SkipAsSource   :   False   PolicyStore   :   ActiveStore 
 
 IPAddress   :   192.168.1.10   InterfaceIndex   :   8   InterfaceAlias   :   Ethernet   2   AddressFamily   :   IPv4  Type   :   Unicast  PrefixLength   :   24   PrefixOrigin   :   Manual  SuffixOrigin   :   Manual  AddressState   :   Invalid  ValidLifetime   :   Infinite   ( [ TimeSpan ] :: MaxValue )   PreferredLifetime   :   Infinite   ( [ TimeSpan ] :: MaxValue )   SkipAsSource   :   False   PolicyStore   :   PersistentStore 

## Cambiar una dirección IP estática

Para cambiar una dirección IP estática existente, tenemos que usar el cmdlet Set-NetIpAddress , pero no es tan fácil. Para cambiar una dirección IP estática existente, primero debe eliminar la existente y crear una nueva. Eliminaré la dirección IP actual y eliminaré la puerta de enlace predeterminada.

In [0]:
 Get-NetIPAddress   -InterfaceAlias   'Ethernet 2'   |   Remove-NetRoute 
 Get-NetIPAddress   -InterfaceAlias   'Ethernet 2'   |   Remove-NetIpAddress 

Una vez que haga esto, usaré la misma técnica que usé anteriormente para agregar una nueva dirección IP.

In [0]:
Get-NetIpAddress   -InterfaceAlias   'Ethernet 2'   |   New-NetIpAddress  ‑ IpAddress  192 . 168 . 1 . 11  -PrefixLength  24  -DefaultGateway  192 . 168 . 1 . 1

Ahora debería ver que la dirección IP ha cambiado

In [0]:
PS   C : \ >   Get - NetIpAddress   - InterfaceAlias   'Ethernet 2' 

In [0]:
 IPAddress           :   192.168.1.11 
 InterfaceIndex      :   8 
 InterfaceAlias      :   Ethernet   2 
 < SNIP > 

# Resumen

Cambiar una dirección IP de NIC y una puerta de enlace predeterminada no es tan sencillo como cabría esperar. Debido a la forma en que Windows almacena la información de IP, descubrirá que eliminar completamente la antigua IP y la puerta de enlace predeterminada y simplemente asignar nuevas a través de los cmdlets es la forma más fácil de realizar esta tarea.